# 

## 1. Data loading and inspection of the manure dataset

In [ ]:
# import libraries
import pandas as pd
import numpy as np
from ydata_profiling import ProfileReport
import matplotlib.pyplot as plt
from scipy import stats

# for static plots
%matplotlib inline

The first step in the process is to load and quickly inspect the main dataset

In [ ]:
# read manure dataset
manure = pd.read_csv('../Data/manure.csv')
# Preview
manure.head()

The preview of the dataset allow us to see that we are dealing with a wide dataframe, that has data related oganic ferilizer name, quantity and unit.

### Insights: 

- The dataset includes a field with household ids useful to identify and correctly identify each datapoint.
- Most of the variable names have capital letters and small letters
- The dataset includes numerical data and categorical data

The second step in the inspection is to show more generalized information on the shape of the dataset (number of variables, number of datapoints), missing values, and data types for each variable.

In [ ]:
manure.info()

### Insights:

- The dataset has 4 variables (columns) and 75 datapoints (rows)
- There are 2 integer data type variables
- There are 2 object (text, boolean, levels, categories) data type variables

## 2. Descriptive statistics of the manure dataset

The following step in the process is to produce a more detailed and thorough statistical profile of the data, that will reveal additional information on potential issues and characteristics of the data.
We use the Ydata libaray that with a simple line of code produces a complete report of descriptive statistics including interactive exploration and automatic visualization.

In [ ]:

profile_manure = ProfileReport(manure, title="Manure Data Profiling Report")
#profile_manure.to_file("report.html")  # HTML report
profile_manure.to_notebook_iframe()

### Insights:
**Numerical**
- OrgFert_Quantity_2024: is the only relevant numerical variable, with a mean organic fertilizer usage of 49.49 units in 2024, it shows a significant standard deviation of 56.73. A maximum value of 400.

**Categorical** 

- Organic fertilizer diversity: Only 3 distinct fertilizer types are used, with Farmyard manure being the dominant one (61% of records).

- Measurement variation: Fertilizer is reported in 6 different units, which may create comparability issues across households.

- Unit dominance: Wheelbarrows is the most common unit (71% of cases), suggesting it’s the standard reference for most farmers.

### Possible data errors/issues:

- hhid duplicated row requires cleaning
- duplicated HHIDs with differing information on other columns

### Oportunities to enhance the data

- Units can be standardized
- fertilizer quantity can be standardized


We proceed to analize possible outliers in the dataset

In [ ]:
manure_no_hhid = manure.drop(columns='HHID')
# plot the numeric variables
plt.figure(figsize=(10, 6))
manure_no_hhid.boxplot()
plt.title('Boxplot of main data')
plt.ylabel('Values')
plt.xticks(rotation=45)
plt.show()

The boxplot shows outliers in the numeric variable.
At this point we don't have additional information to contextualize the values in the dataset.

In [ ]:
#zscore
outlier_df = pd.concat([manure[(np.abs(stats.zscore(manure[col].dropna())) > 2).reindex(manure.index, fill_value=False)].assign(outlier_column=col) for col in manure.select_dtypes(include=[np.number])])
outlier_df[outlier_df['outlier_column']== 'OrgFert_Qantity_2024']

Using the Z-score statistic, we don't find outliers. We will contextualize the values better once we merge the main and manure dataset.

## 3. Data cleaning and transformation

The first step in the data cleaning and transformation is to create a copy of the original dataset, to ensure the changes can be easlily reverted in case of issues. After that and follwing the insights from previous steps we will start by standarizing the string of column names

In [ ]:
manure.columns = manure.columns.str.lower()
print(manure.columns.tolist())

The next step is to remove the duplicated identical rows

In [ ]:
# Remove duplicated rows
manure_std = manure.copy()

# Check for duplicates before removal
duplicates_before = manure_std.duplicated().sum()
print(f"Number of duplicate rows found in original dataset: {duplicates_before}")

if duplicates_before > 0:
    print("\nSample of duplicate rows:")
    print(manure_std[manure_std.duplicated()].head())
else:
    print("No duplicate rows found in the original dataset.")

# Remove duplicates
manure_std = manure_std.drop_duplicates()

# Check for duplicates after removal
duplicates_after = manure_std.duplicated().sum()
print(f"\nNumber of duplicate rows after removal: {duplicates_after}")

# Print final shape comparison
print(f"\nDataset shape change:")
print(f"Rows removed: {len(manure) - len(manure_std)}")

We continue by standardizing the organic fertilizer units, create a conversion factor map for units, and finally calculate the standarized  fertilization quantities in Kilograms

In [ ]:
# see unit values in the dataframe
print("Unique values in orgfert_unit_2024 column:")
print(manure_std['orgfert_unit_2024'].unique())

# Create a standardized version of the unit column for matching
manure_std['orgfert_unit_2024_std'] = manure_std['orgfert_unit_2024'].astype(str).str.lower().str.strip()

# preview cleaned values
print("\nCleaned unique values:")
print(manure_std['orgfert_unit_2024'].unique())

# Conversion factors based on the cleaned units
conversion_factors_manure = {
    'kg': 1,
    '25 kg bag': 25,
    'bucket/debe': 20, # weigth estimated for a standard 20L bucket
    'ton': 1000,
    '90 kg bag': 90,
    'wheelbarows':40.  # weigth estimated for a standard 65L wheelbarow
}

# Apply conversion using the cleaned column
manure_std['orgfert_quantity_2024_kg'] = manure_std['orgfert_quantity_2024'] * manure_std['orgfert_unit_2024_std'].map(conversion_factors_manure)

# Check for any missing values (units not in our conversion map)
missing_mask = manure_std['orgfert_quantity_2024_kg'].isna()
if missing_mask.any():
    print(f"\nWarning: {missing_mask.sum()} records had units not found in conversion map:")
    print(manure_std[missing_mask]['orgfert_unit_2024_std'].unique())
    
    # Fill missing values with original quantity (or you can choose another strategy)
    manure_std.loc[missing_mask, 'orgfert_quantity_2024_kg'] = manure_std.loc[missing_mask, 'orgfert_quantity_2024']

# Check the results
print("\nConversion results:")
manure_std.tail(10)

The next step is to analize and address the duplicated household IDs with diferring information of fertilizer quantities and types

In [ ]:
duplicates_manure = manure[manure.duplicated(subset=['hhid'], keep=False)]
print(duplicates_manure)

After checking the duplicated rows, we infer that this is not likely an error, but that it could represent multiple reports from the same households. I.e. 2 rounds of fertilization.

As at this point it is imposible to confirm with the data collection or field teams, we make the assuption that data is representing 2 rounds of applications and then proceed to agreegate the quantities by adding them. 

To keep record of the original values, we create new columns where:

- orgfert_qty_2024_kg_total: is the sum of fertilizer quantities, if household is not duplicated the sum will be equal to orgfert_qty_2024_kg

- orgfert_names_2024: a string of the fertilizer(s) reported

- orgfert_qty_2024_kg: a list of fertilizer(s) quantities applied


In [ ]:
manure_std_cons = manure_std.groupby("hhid").agg(
    orgfert_qty_2024_kg_total=("orgfert_quantity_2024_kg", "sum"),
    orgfert_names_2024=("orgfert_name_2024", lambda x: ", ".join(map(str, x))),
    orgfert_qty_2024_kg=("orgfert_quantity_2024_kg", lambda x: ", ".join(map(str, x)))
).reset_index()
manure_std_cons.head()

To finalize, we run the descritive statics report to check results

In [ ]:
manure_stats_cols = ['orgfert_qty_2024_kg_total','orgfert_names_2024']
profile_manure_cons = ProfileReport(manure_std_cons[manure_stats_cols], title="Manure Standarized Data Profiling Report")
#profile_manure.to_file("report.html")  # HTML report
profile_manure_cons.to_notebook_iframe()

The last step in the analisys is to save the cleaned manure data set as csv file

In [ ]:
manure_std_cons.to_csv('../data/cleaned/manure_std_cons.csv', index=False)